In [1]:
import os
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import LeaveOneOut, KFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

## New data

In [2]:
label_dict = {4:0, 5:1}

In [3]:
X_train = pd.read_csv("../data/HAPT/Train/X_train.txt", header=None, sep=" ")
y = pd.read_csv("../data/HAPT/Train/y_train.txt", header=None).rename(columns={0:"label"})
X_train["label"] = y.label
X_train = X_train.sample(frac=1.).reset_index(drop=True)

X_test = pd.read_csv("../data/HAPT/Test/X_test.txt", header=None, sep=" ")
y = pd.read_csv("../data/HAPT/Test/y_test.txt", header=None).rename(columns={0:"label"})
X_test["label"] = y.label 

In [4]:
X_train.shape, X_test.shape

((7767, 562), (3162, 562))

In [5]:
np.unique(X_train.label, return_counts=True)

(array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12]),
 array([1226, 1073,  987, 1293, 1423, 1413,   47,   23,   75,   60,   90,
          57]))

In [6]:
train_idx = X_train.label.isin([4,5])
np.unique(X_train[train_idx].label, return_counts=True)

(array([4, 5]), array([1293, 1423]))

In [7]:
# Sitting vs standing
train_idx = X_train.label.isin([4,5])
X_train = X_train[train_idx].reset_index(drop=True)
X_train["label"] = X_train["label"].map(label_dict)

test_idx = X_test.label.isin([4,5])
X_test = X_test[test_idx].reset_index(drop=True)
X_test["label"] = X_test["label"].map(label_dict)
y_test = X_test["label"]

In [8]:
X_train.shape, X_test.shape

((2716, 562), (1064, 562))

In [9]:
np.unique(X_train.label, return_counts=True)

(array([0, 1]), array([1293, 1423]))

In [10]:
X_train.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511,512,513,514,515,516,517,518,519,520,521,522,523,524,525,526,527,528,529,530,531,532,533,534,535,536,537,538,539,540,541,542,543,544,545,546,547,548,549,550,551,552,553,554,555,556,557,558,559,560,label
0,0.038216,-0.005712,-0.021914,-0.998348,-0.995431,-0.991390,-0.998184,-0.995155,-0.992187,-0.802836,-0.750945,-0.718481,0.840957,0.714987,0.676839,-0.996569,-0.999992,-0.999961,-0.999906,-0.997285,-0.995487,-0.991556,-0.867607,-0.796640,-0.642749,0.217886,-0.085428,-0.018600,0.231355,0.230877,-0.180679,0.130887,-0.000333,0.296732,-0.093821,0.079359,-0.121715,0.440605,0.375685,0.309766,0.923927,-0.307083,-0.059027,-0.997500,-0.995701,-0.991339,-0.997468,-0.995527,-0.991098,0.853679,-0.330423,-0.064355,0.947448,-0.280199,-0.058973,-0.097368,0.797939,-0.832331,-0.991688,-0.997583,-0.995815,-0.991202,-1.000000,-1.0,-1.000000,-0.570067,0.578585,-0.587140,0.595712,-0.607722,0.613870,-0.620420,0.626455,-0.247365,0.249265,-0.251013,0.251926,0.953591,0.998713,0.949214,0.073640,0.008935,-0.032472,-0.998118,-0.996145,-0.992440,-0.998620,-0.995296,-0.991194,-0.996954,-0.998805,-0.991227,0.995762,0.995167,0.992181,-0.997500,-0.999985,-0.999955,-0.999860,-0.998042,-0.993187,-0.988137,-0.875096,-0.982792,-0.758016,0.192313,-0.060831,0.239542,0.054918,0.294881,0.041307,0.245196,0.428557,0.217915,0.149598,0.174465,0.408847,0.251277,0.470362,-0.165974,-0.028714,-0.020631,-0.077878,-0.995705,-0.993694,-0.995973,-0.995554,-0.993493,-0.995635,-0.910959,-0.957273,-0.771712,0.849056,0.893793,0.827817,-0.994533,-0.999984,-0.999964,-0.999977,-0.995050,-0.993712,-0.994759,-0.633226,-0.464965,-0.625352,0.318084,-0.354852,0.321817,-0.121066,-0.019190,-0.021307,0.032425,0.154276,0.374829,-0.238999,0.140813,0.093786,-0.537437,-0.036864,-0.371105,-0.098837,-0.038925,-0.057068,-0.997669,-0.998675,-0.996074,-0.997560,-0.998607,-0.996214,-0.997861,-0.997759,-0.993970,0.998340,0.999236,0.996323,-0.998641,-0.999988,-0.999996,-0.999969,-0.997422,-0.997978,-0.996169,-0.826652,-0.866808,-0.718448,0.342422,-0.094850,0.211233,0.115510,0.083996,0.031662,0.152799,-0.00

# Logreg

In [11]:
def get_predictions(x):
    return [1 if xi >= 0.5 else 0 for xi in x]

In [12]:
X, y = X_train[[col for col in X_train.columns if col != "label"]].values , X_train.label

In [13]:
X.shape, y.shape

((2716, 561), (2716,))

In [15]:
loo = KFold(n_splits=10)
preds = np.zeros(len(y))
for i, (train_index, test_index) in enumerate(loo.split(X)):
    _X_train, _X_test = X[train_index, :], X[test_index,:]
    _y_train = y[train_index].values
    clf = LogisticRegression(random_state=0, C=1., max_iter=500).fit(_X_train, _y_train)
    preds[test_index] = clf.predict_proba(_X_test)[:,1]

print(f"Models AUC score: {roc_auc_score(y, preds)}")
print(classification_report(y, get_predictions(preds)))

Models AUC score: 0.9930524870661473
              precision    recall  f1-score   support

           0       0.96      0.96      0.96      1293
           1       0.96      0.96      0.96      1423

    accuracy                           0.96      2716
   macro avg       0.96      0.96      0.96      2716
weighted avg       0.96      0.96      0.96      2716



In [16]:
# Logreg MLE
log_reg = LogisticRegression(random_state=0, C=1.0, max_iter = 500).fit(X[:561,:], y[:561])

In [17]:
_X_test = X_test[[col for col in X_test.columns if col != "label"]].values
y_pred = log_reg.predict_proba(_X_test)[:,1]

In [18]:
print(f"Models AUC score: {roc_auc_score(y_test, y_pred)}")

Models AUC score: 0.974823684359599


# Gaussian logreg

## MAP l2 estimate C=0.3

In [19]:
loo = KFold(n_splits=10)
preds = np.zeros(len(y))
for i, (train_index, test_index) in enumerate(loo.split(X)):
    _X_train, _X_test = X[train_index, :], X[test_index,:]
    _y_train = y[train_index].values
    clf = LogisticRegression(random_state=0, C=.3, max_iter=500).fit(_X_train, _y_train)
    preds[test_index] = clf.predict_proba(_X_test)[:,1]

print(f"Models AUC score: {roc_auc_score(y, preds)}")
print(classification_report(y, get_predictions(preds)))

Models AUC score: 0.9914823263162529
              precision    recall  f1-score   support

           0       0.95      0.96      0.95      1293
           1       0.96      0.96      0.96      1423

    accuracy                           0.96      2716
   macro avg       0.96      0.96      0.96      2716
weighted avg       0.96      0.96      0.96      2716



In [20]:
log_reg = LogisticRegression(random_state=0, C=.3, penalty="l2", max_iter = 500).fit(X[:561,:], y[:561])

In [21]:
_X_test = X_test[[col for col in X_test.columns if col != "label"]].values
y_pred = log_reg.predict_proba(_X_test)[:,1]
print(f"Models AUC score: {roc_auc_score(y_test, y_pred)}")

Models AUC score: 0.9766293264600917


# Slap-and-spike prior

In [29]:
X, y = X_train[[col for col in X_train.columns if col != "label"]].values , X_train.label

In [30]:
import pymc3 as pm
import theano as tt
from scipy.special import expit
from scipy.stats import norm, bernoulli

This is our slab and spike model

$$a \sim \mathcal{N}(0, 3)$$
$$\gamma_i \sim Bernoulli(p=0.1)$$
$$\alpha_i|\sigma_\beta \sim \mathcal{N}(0, \sigma_\beta)$$
$$e \sim \mathcal{N}(0, \sigma^2_eI_n)$$
$$y \sim \frac{1}{1+exp(-(a + \sum_{i=1}^N \gamma_i \alpha_i x_i + e))}$$

The model parameters are $\theta = \{\gamma_i, \alpha_i\}_i^N $. The $\gamma_i$ and $\alpha_i$ are modelled IID.

In [31]:
prob = 0.1
a_mu = 0
a_var = 3
gamma_var = 1
with pm.Model() as model:
    # priors inclusion probability
    gamma_i = pm.Bernoulli("gamma_i", prob, shape=X.shape[1])
    # a is the interception
    a = pm.Normal("a", mu=a_mu, sd=a_var)
    # The prior for the features varibles which are included
    alpha = pm.Normal("alpha", mu=0, sd=gamma_var, shape=X.shape[1])
    # Deterministic function
    p = pm.math.dot(X,gamma_i * alpha) 
    # Likelihood
    y_obs = pm.Bernoulli("y_obs", pm.invlogit(p + a),  observed=y)
 

In [32]:
with model:
    trace = pm.sample(2000, random_seed = 4816, cores = 1, progressbar = True, chains = 1)
    

/var/folders/bx/2b_70g054qg3qt5jhnw30xv40000gn/T/ipykernel_47018/2955332559.py:2: FutureWarning: In v4.0, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
  trace = pm.sample(2000, random_seed = 4816, cores = 1, progressbar = True, chains = 1)
Sequential sampling (1 chains in 1 job)
CompoundStep
>BinaryGibbsMetropolis: [gamma_i]
>NUTS: [alpha, a]


Sampling 1 chain for 1_000 tune and 2_000 draw iterations (1_000 + 2_000 draws total) took 2142 seconds.
There were 24 divergences after tuning. Increase `target_accept` or reparameterize.
Only one chain was sampled, this makes it impossible to run some convergence checks


# Map estimate

The log loss for the spike and slap prior
$$p(\theta) = p(\{\gamma_i, \alpha_i\}_i^N) = \prod_{i=1}^N p(\gamma_i) p(\alpha_i)$$

$$\log p(\theta) = \sum_{i=1}^N \log Bernoulli(\gamma_i | p=0.1) + \log \mathcal{N}(\alpha_i | \mu=0, \sigma_\beta=3)$$

In [33]:
def spike_slab_log_prior(gamma: np.array, alpha: np.array, p, gamma_mu=0, sigma_beta=3):
    return (bernoulli.logpmf(gamma, p=p) + norm.logpdf(alpha, loc=gamma_mu, scale=sigma_beta)).sum()

The negative log likelihood function, i.e. cross entropy

$$E(\mathbf{w}) = -log(p(\pmb{\mathbf{t}}|\pmb{\mathbf{x}},\mathbf{w})) \nonumber$$

$$= - \sum_{n=1}^N \left( t_n \ln y(\mathbf{x}_n) + (1-t_n) \ln (1-y(\mathbf{x}_n)) \right)$$
  

In [34]:
def log_likelihood(a, gamma, alpha, X, T):
    y_x = expit(a + np.dot(X, np.transpose(gamma*alpha)))
    return (T*np.log(y_x) + ((1-T)*np.log(1-y_x))).sum()

In [35]:
prob = 0.1
gamma_mu = 0
sigma_beta = 3

def find_spike_slab_MAP(trace, X, y, prob, gamma_mu, sigma_beta):
    min_loss = np.inf
    cur_min = -1
    for i in range(len(trace)):
        tmp_trace = trace[i]
        tmp_spike_slab_log_prior = spike_slab_log_prior(tmp_trace["gamma_i"], tmp_trace["alpha"], p=prob, gamma_mu=gamma_mu, sigma_beta=sigma_beta)
        tmp_log_likelihood = log_likelihood(tmp_trace["a"], tmp_trace["gamma_i"], tmp_trace["alpha"], X, y)
        neq_loss = -(tmp_log_likelihood + tmp_spike_slab_log_prior)
        if neq_loss <= min_loss:
            min_loss = neq_loss
            cur_min = i
    return trace[cur_min]
    

In [36]:
map_trace = find_spike_slab_MAP(trace, X, y, prob, gamma_mu, sigma_beta)

In [37]:
map_estimate = map_trace["gamma_i"] * map_trace["alpha"]

_X_test = X_test[[col for col in X_test.columns if col != "label"]].values
map_preds = expit(map_trace["a"] + np.dot(_X_test, np.transpose(map_estimate)))

In [47]:
print(f"Models AUC score: {roc_auc_score(y_test, map_preds)}")

Models AUC score: 0.9642659887837761


# Full Bayesian

In [39]:
results = pd.DataFrame({'var': np.arange(561), 
                        'inclusion_probability':np.apply_along_axis(np.mean, 0, trace['gamma_i']),
                       'alpha':np.apply_along_axis(np.mean, 0, trace['alpha']),
                       'alpha_given_inclusion': np.apply_along_axis(np.sum, 0, trace['gamma_i']*trace['alpha'])
                            /np.apply_along_axis(np.sum, 0, trace['gamma_i'])
                       })

In [40]:
results.sort_values('inclusion_probability', ascending = False).head(10)


,var,inclusion_probability,alpha,alpha_given_inclusion
41,41,1.0000,-3.883960,-3.883960
451,451,1.0000,-2.267181,-2.267181
182,182,1.0000,5.423177,5.423177
142,142,1.0000,1.571083,1.571083
50,50,1.0000,-4.035479,-4.035479
445,445,1.0000,4.380210,4.380210
186,186,1.0000,-3.663025,-3.663025
187,187,0.9970,-2.154221,-2.158809
158,158,0.9970,-0.996644,-1.002019
53,53,0.9915,-3.449169,-3.479308


## Bayesian inference

In [41]:
estimate = trace['alpha'] * trace['gamma_i'] 
_X_test = X_test[[col for col in X_test.columns if col != "label"]].values
preds = np.apply_along_axis(np.mean, 1, expit(trace['a'] + np.dot(_X_test, np.transpose(estimate) )) )


In [42]:
print(f"Models AUC score: {roc_auc_score(y_test, preds)}")

Models AUC score: 0.9731986064691553


In [43]:
preds[:10]

array([0.99728329, 0.95677686, 0.92627129, 0.96335403, 0.97437116,
       0.94798007, 0.89826115, 0.83813811, 0.84503151, 0.9424179 ])

In [44]:
map_preds[:10]

array([0.99377665, 0.9638406 , 0.86181986, 0.90449025, 0.98654351,
       0.9787043 , 0.94709745, 0.91942937, 0.89949213, 0.92160971])

In [45]:
(map_preds[map_preds > 0.5]).mean(), (map_preds[map_preds <= 0.5]).mean()

(0.9363551762776625, 0.09595969172622476)

In [46]:
(preds[preds > 0.5]).mean(), (preds[preds <= 0.5]).mean()

(0.9245190406644662, 0.09512387902157081)